In [7]:
import torch
import torch.nn as nn
from GPT_module import TransformerBlock, LayerNorm


In [8]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,  # vocabsize from BPE tokenizer
    "context_length": 1024,  # context length
    "emb_dim": 768,  # embedding dimension
    "n_heads": 12,  # number of attention heads
    "n_layers": 12,  # number of layers
    "drop_rate": 0.1,  # dropout rate
    "qkv_bias": False  # query-key-value bias
}


**GPT Model**

In [9]:


class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb=nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb=nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb=nn.Dropout(cfg["drop_rate"])

        self.trf_blocks=nn.Sequential(
            *[TransformerBlock(cfg=cfg) for _ in range(cfg["n_layers"])]
        )

        self.final_norm=LayerNorm(cfg["emb_dim"])
        self.out_head=nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )



    def forward(self, in_idx):
        batch_size, seq_len=in_idx.shape
        tok_embeds=self.tok_emb(in_idx)
        pos_embeds=self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x=tok_embeds+pos_embeds
        x=self.drop_emb(x)
        x=self.trf_blocks(x)
        x=self.final_norm(x)
        logits=self.out_head(x)
        return logits





In [10]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
batch = []
txt1 = 'Every effort moves you'
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)

print(batch)  # token IDs for 2 texts
print(batch.shape)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
torch.Size([2, 4])


In [11]:
torch.manual_seed(123)
model=GPTModel(GPT_CONFIG_124M)

out=model(batch)
print(out.shape)
print(out)


torch.Size([2, 4, 50257])
tensor([[[ 0.3613,  0.4222, -0.0711,  ...,  0.3483,  0.4661, -0.2838],
         [-0.1792, -0.5660, -0.9485,  ...,  0.0477,  0.5181, -0.3168],
         [ 0.7120,  0.0332,  0.1085,  ...,  0.1018, -0.4327, -0.2553],
         [-1.0076,  0.3418, -0.1190,  ...,  0.7195,  0.4023,  0.0532]],

        [[-0.2564,  0.0900,  0.0335,  ...,  0.2659,  0.4454, -0.6806],
         [ 0.1230,  0.3653, -0.2074,  ...,  0.7705,  0.2710,  0.2246],
         [ 1.0558,  1.0318, -0.2800,  ...,  0.6936,  0.3205, -0.3178],
         [-0.1565,  0.3926,  0.3288,  ...,  1.2630, -0.1858,  0.0388]]],
       grad_fn=<UnsafeViewBackward0>)


In [12]:
# total number of parameters
total_params=sum(p.numel() for p in model.parameters())
print(f"{total_params:,}")

163,009,536


In [13]:
# the memory requirment for 163 million parameters
total_size_bytes=total_params*4
total_size_mb=total_size_bytes/(1024*1024)
print(total_size_mb)

621.83203125


**A function for GPT model to generate text**

In [20]:
def generate_text_simple(model, idx, max_new_tokens, context_size): # idx: (batch, n_tokens) array of indices in the current context

    for _ in range(max_new_tokens):
        idx_cond=idx[:,-context_size:] # crop the current context if it exceed supported context
        with torch.no_grad():
            logits=model(idx_cond)

        logits=logits[:,-1,:] # focusing last step
        probs=torch.softmax(logits, dim=-1)
        idx_next=torch.argmax(probs, dim=-1, keepdim=True)
        idx=torch.cat((idx,idx_next), dim=1)

    return idx



In [ ]:
# testing the gpt model
import tiktoken
tokenizer=tiktoken.get_encoding("gpt2")

start_context="Hello, I am"
encoded=tokenizer.encode(start_context)
print(encoded) # encoded stings

encoded_tensor=torch.tensor(encoded).unsqueeze(0) # unsqueeze() add extra dimentionality
print(encoded_tensor)
print(encoded_tensor.shape) # 1 batch size, 4 number of tokens




[15496, 11, 314, 716]
tensor([[15496,    11,   314,   716]])
torch.Size([1, 4])


In [30]:
model.eval() # testing mode
out=generate_text_simple(model=model,idx=encoded_tensor, max_new_tokens=6, context_size=GPT_CONFIG_124M["context_length"])
print(out)
print(len(out[0]))



tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
10


In [ ]:
# now from tokenizer, we can use decode to decode it back
decoded_text=tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)


Hello, I am Featureiman Byeswickattribute argue


In [14]:
# GPT_CONFIG_MEDIUM = {
#     "vocab_size": 50257,  # vocabsize from BPE tokenizer
#     "context_length": 1024,  # context length
#     "emb_dim": 1024,  # embedding dimension
#     "n_heads": 16,  # number of attention heads
#     "n_layers": 24,  # number of layers for transformer block
#     "drop_rate": 0.1,  # dropout rate
#     "qkv_bias": False  # query-key-value bias
# }


In [15]:
# torch.manual_seed(123)
# model_medium=GPTModel(GPT_CONFIG_MEDIUM)
# total_params = sum(p.numel() for p in model_medium.parameters())
# print(f"{total_params:,}")


In [16]:
# GPT_CONFIG_LARGE = {
#     "vocab_size": 50257,  # vocabsize from BPE tokenizer
#     "context_length": 1024,  # context length
#     "emb_dim": 1280,  # embedding dimension
#     "n_heads": 20,  # number of attention heads
#     "n_layers": 36,  # number of layers for transformer block
#     "drop_rate": 0.1,  # dropout rate
#     "qkv_bias": False  # query-key-value bias
# }


In [17]:
# torch.manual_seed(123)
# model_large = GPTModel(GPT_CONFIG_LARGE)
# total_params = sum(p.numel() for p in model_large.parameters())
# print(f"{total_params:,}")


In [18]:
# GPT_CONFIG_XL = {
#     "vocab_size": 50257,  # vocabsize from BPE tokenizer
#     "context_length": 1024,  # context length
#     "emb_dim": 1600,  # embedding dimension
#     "n_heads": 25,  # number of attention heads
#     "n_layers": 48,  # number of layers for transformer block
#     "drop_rate": 0.1,  # dropout rate
#     "qkv_bias": False  # query-key-value bias
# }


In [19]:
# torch.manual_seed(123)
# model_xl = GPTModel(GPT_CONFIG_XL)
# total_params = sum(p.numel() for p in model_xl.parameters())
# print(f"{total_params:,}")
